<a href="https://colab.research.google.com/github/jcmachicao/gdmk_uc__reportes_edusights/blob/main/ucont__mprecitivo_train_202502_tensorready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de Red Neuronal Clasificación Binaria PyTorch

## Preparación

In [ ]:
!pip install torch torchvision

https://docs.neptune.ai/setup/upgrading/

In [ ]:
!pip install -q neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torchvision import datasets, transforms
from google.colab import userdata
import os

In [ ]:
if torch.cuda.is_available():
    print("GPU is available.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available.
Device name: Tesla T4


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!nvidia-smi

Mon Feb 24 17:53:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
data_path = 'drive/My Drive/00 Aplicativos/data_aplicativos/archivos/'
os.listdir(data_path)

['df_Ingresantes_uvir_2220al2410_0107.pkl',
 'df_Ingresantes_uvir_2220al2410_abandono_mas_deserción.pkl',
 'df_Ingresantes_uvir_2220al2410_deserción.pkl',
 'Estudiantes_202420_al1107.xlsx',
 'X_train.pt',
 'y_train.pt',
 'X_test.pt',
 'y_test.pt']

## Rerun or Training

In [ ]:
X_tot = torch.load(data_path + 'X_train_tensor.pt')
y_tot = torch.load(data_path + 'y_train_tensor.pt')
X_tot.shape, y_tot.shape

(torch.Size([8020, 149]), torch.Size([8020]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tot, y_tot, test_size=FRAC_TEST, random_state=None)

In [ ]:
torch.save(X_train, data_path + 'archivos/X_train.pt')
torch.save(y_train, data_path + 'archivos/y_train.pt')
torch.save(X_test, data_path + 'archivos/X_test.pt')
torch.save(y_test, data_path + 'archivos/y_test.pt')

In [ ]:
X_train_tensor = X_train.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
X_test_tensor = X_test.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
y_train_tensor = y_train.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
y_test_tensor = y_test.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True

In [ ]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape, y_train_tensor.shape)
print(X_test_tensor.shape, y_test_tensor.shape)

torch.Size([6416, 149]) torch.Size([6416])
torch.Size([1604, 149]) torch.Size([1604])


In [ ]:
class ImprovedNN1(nn.Module):
    def __init__(self, input_size):
        super(ImprovedNN1, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))
        return x


### Tensores

In [ ]:
input_size = X_train.shape[1]
num_classes = len(np.unique(y_tot))
print('Tamaño de Entrada: ', input_size, 'Número de Clases de Salida: ', num_classes)
model = ImprovedNN1(input_size).to(device)

Tamaño de Entrada:  149 Número de Clases de Salida:  2


In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
criterion = nn.BCELoss()  # or nn.BCEWithLogitsLoss() if you remove the sigmoid from the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

def train_model(model, train_loader, criterion, optimizer, num_epochs=NUM_EPOCHS):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.squeeze()  # Squeeze to remove extra dimensions
            loss = criterion(outputs, labels.float())  # Ensure labels are floats
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        run["train/loss"].log(epoch_loss)  # Ensure 'run' is defined if you are using it for logging
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:
def evaluate_model_v2(model, test_data, test_labels):
    model.eval()
    with torch.no_grad():
        outputs = model(test_data)

    outputs_show = outputs.cpu().numpy().flatten()
    outputs_show[outputs_show > 0.51] = 1.0
    outputs_show[outputs_show < 0.49] = 0.0
    test_labels_np = test_labels.detach().cpu().numpy()

    accuracy = 100 * (np.sum(outputs_show == test_labels_np) / len(test_labels_np))
    print(f'Precisión del Modelo en Data de Prueba: {accuracy:.2f}%')

    results_df = pd.DataFrame({
        'Real': test_labels_np,
        'Pred': outputs_show
    })

    return results_df, accuracy

### Training

In [ ]:
# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)

Epoch [310/800], Loss: 0.0177
Epoch [311/800], Loss: 0.0241
Epoch [312/800], Loss: 0.0207
Epoch [313/800], Loss: 0.0207
Epoch [314/800], Loss: 0.0258
Epoch [315/800], Loss: 0.0217
Epoch [316/800], Loss: 0.0175
Epoch [317/800], Loss: 0.0205
Epoch [318/800], Loss: 0.0181
Epoch [319/800], Loss: 0.0204
Epoch [320/800], Loss: 0.0241
Epoch [321/800], Loss: 0.0194
Epoch [322/800], Loss: 0.0151
Epoch [323/800], Loss: 0.0183
Epoch [324/800], Loss: 0.0230
Epoch [325/800], Loss: 0.0136
Epoch [326/800], Loss: 0.0184
Epoch [327/800], Loss: 0.0250
Epoch [328/800], Loss: 0.0187
Epoch [329/800], Loss: 0.0135
Epoch [330/800], Loss: 0.0164
Epoch [331/800], Loss: 0.0198
Epoch [332/800], Loss: 0.0210
Epoch [333/800], Loss: 0.0229
Epoch [334/800], Loss: 0.0177
Epoch [335/800], Loss: 0.0197
Epoch [336/800], Loss: 0.0207
Epoch [337/800], Loss: 0.0188
Epoch [338/800], Loss: 0.0183
Epoch [339/800], Loss: 0.0198
Epoch [340/800], Loss: 0.0228
Epoch [341/800], Loss: 0.0192
Epoch [342/800], Loss: 0.0202
Epoch [343

In [ ]:
results_df, accuracy = evaluate_model_v2(model, X_test_tensor, y_test_tensor)
results_df.to_excel(data_path + f'results_{version_name}.xlsx')

Precisión del Modelo en Data de Prueba: 58.85%
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/jcmachicao/continental-edusights/e/CON-69/metadata


In [ ]:
accuracy

58.85286783042394

https://sites.google.com/continental.edu.pe/edusights/ModVIDAD?authuser=0

In [ ]:
model_name = f'edusights_20240702_model_{version_name}.pth'
dict_name = f'edusights_20240702_state_dict_{version_name}.pth'
torch.save(model, data_path + 'modelos/' + model_name)
torch.save(model.state_dict(), data_path + 'modelos/' + dict_name)